In [430]:
#//----------------------------
#//LIBRARIES
    #Math
import math
    #Numeric Python
import numpy as np
    #Pandas (dataframes)
import pandas as pd
    #datetime for fate manipulation
from datetime import date, datetime, timedelta  
    #Regex for advanced string matching
import re
    #for time related stuff
import time
    #json library
import json
    #Analyst tools
import sys
sys.path.append('../')
from analysts_tools.growth import *
    #Procurement tools
from procurement_lib import send_slack_notification
from procurement_lib import redash
from analysts_tools.redash_methods import *

import random
import time

In [431]:
#//----------------PRECARGA-------------------------//
#//Ruta de data // Base Data (Without Deleting) 
segmentos_fulll = get_fresh_query_result("https://internal-redash.federate.frubana.com/",121983,'SeoGHWmDUaaBi7VXje1s9zYNiMD1VHQ1K1DYOxiF',{},20)
segmentos_fulll.columns = ['customer_id', 'city', 'microsegment','NA']

200


In [432]:
segmentos_fulll = segmentos_fulll[['customer_id', 'city', 'microsegment']]

In [433]:
#//---------------------------------------------------------------------------------------------------------------------------
#// Parametros
"""Todays date should be the first day of the next month which is going to begin with SD"""
city_code = 'CMX'
    #year-month-day
todays_date = '2023-01-01' 
non_buyers_date = '2022-12-01' #1 months
customer_date_desde = '2022-12-01' #3 months
    #data for sku sales data
sku_date_desde = '2022-11-01' #2 months

t_cambio = 1 #COL=3000 // MX=19.19 // BR=3.88

### Loading COSTUMERS INFO

In [434]:
#//--------------------------------------
#//Ruta de data // Base Data (Without Deleting)
if city_code in ('BAQ', 'BOG', 'CMX', 'SPO','MDE','GDL','PBC','CWB','BHZ','VCP'):
    segmentos_full = segmentos_fulll[segmentos_fulll.city == city_code].copy()
else:
    data = [[city_code,'NA',0]]
    segmentos_full = pd.DataFrame(data, columns=['city', 'microsegment', 'customer_id'])

In [435]:
#//--------------------------------------
#//Ruta de data // KAM's costumers to delete
#//KAMs customers me lo manda Emiliano Lomeli (SE ACTUALIZAN DE UN SCRIPT MES A MES)
users_eliminar_kam = pd.read_csv("SD/import_data/segments/KAMs_NOV.csv")['costumer_id'].unique().tolist()

In [436]:
#//Users to ignore  ->>> ACAA [(ACA IMPORTANTE VER EL SEGMENTOS_PREV2 QUE ESTAS TRAYENDO)]
#//At the start of the month we just delete KAM users, then Users that already got SD
users_eliminar = users_eliminar_kam #+ segmentos_prev1['customer_id'].unique().tolist() #+ segmentos_prev2['customer_id'].unique().tolist() + segmentos_prev3['customer_id'].unique().tolist()

### Loading FORCED_SKUS // Cleaning SEGMENT_NAMES

In [437]:
data = [['NA','NA','NA',0]]
forced_skus = pd.DataFrame(data, columns=['microsegment', 'subcat', 'name', 'sku_ids'])

### Loading DATA_VENTAS // Removing SKUs /\ CUSTOMERS_ID

In [438]:
#//Remove skus 
#It is used because some SKUs can get in trouble with supplier for the SD or to exclude them for another activity 
skus_remove = [
166212,
283894,
290295,
166210,
165914,
175066,
165916,
94250,
231534,
105657,
175054,
165866,
290296,
283896,
175068,
198505,
165899,
232256,
165909,
283895,
232355,
165922,
165924,
165930,
232350,
165932,
179696,
256705,
256693,
173947,
53047,
179697,
-301275,
274988,
312799,
192151,
-300546,
256511,
274987,
192150,
256512,
-301275,
192147,
192148,
256694,
173946,
168100,
168101,
168100,
232345,
165938,
165940,
165907,
232365,
165946,
166222,
441216,
165901,
232257,
165948,
198505,
165868,
198495,
232370,
175056,
232339,
165882,
165884,
441216,
441217,
166054,
166056,
232360, #HASTA ACA
262884,
262885,
262886,
443352,
443351,
271395,
271396,
271392,
99017,
324145,
324143,
56606,
56612,
-301818, #Diana
124661, #Diana
124659, #Diana
56612, #alquería
401540, #alquería 
56606, #alquería
90176, #alquería
266755, #alquería
126505, #alquería
#### experimento mayo
-303030,
124708,
124710,
-303057,
124702,
124700,
124716,
63697,
280865,
280868,
280862,
440367,
-304254,
-302913,
277542,
-300543,
62912,
212438,
-302130,
210161,
210159,
97137,
83969,
241989,
271594,
271602,
147339,
254113,
254112,
31507,
82996,
102355,
111139,
118025,
118027,
118020,
-304413,
48733,
-302271,
-302286,
367833,
136080,
136082,
130690,
290511,
169189,
290224,
290223,
290222,
377870,
377869,
377868,
219702,
37211,
219697,
37188,
296040,
296041,
91360,
91356,
106890,
106889,
106895,
162844,
162845,
162843,
278268,
322342,
278267,
285339,
285338,
195864,
195961,
195960,
290048,
290049,
290047,
218839,
218840,
218841,
197104,
197105,
359613,
196789,
196790,
196791,
250239,
250240,
197116,
342968,
342969,
342967,
197114,
197113,
197718,
197719,
197720,
195993,
195992,
195994,
304504,
249435,
249433,
249434,
290283,
371569,
371567,
371571,
371573,
254974,
254842,
254971,
254844,
254865,
254856,
295085,
254973,
336632,
249954,
249953,
249993,
249992,
404804,
404799,
285549,
285348,
285347,
296190,
285353,
285548,
285552,
285551,
285354,
285345,
332389,
296188,
296189,
285344,
332390,
126139,
491240,
191539,
1207,
490568,
1209,
1265,
123039,
491238,
333292,
333294,
333296,
333298,
489261,
333300,
333301,
323989,
323990,
323995,
323996,
323998,
323999,
322945,
322946,
489263,
366572,
366573,
366575,
271393,
367096,
367101,
491189,
489257,
490566,
147341,
489259,
489253,
490567,
367142,
491188,
120813,
491239,
203539,
323801,
323809,
588947
]

In [439]:
frida_products = get_fresh_query_result("https://internal-redash.federate.frubana.com/",48322,'SeoGHWmDUaaBi7VXje1s9zYNiMD1VHQ1K1DYOxiF',{},20)

200


In [440]:
good_skus = frida_products.sku_id.unique()

In [441]:
def detectSKUs(skus, strs):
    
    if  str(strs)=='nan':
        return True #To exclude nans    
    for sku in skus:
        if str(sku) in strs:
            return True
    return False

In [442]:
#Loading SELL'S DATA from a function
data_ventas = data_ventas_query(ciudad=city_code, tipo_cambio=t_cambio, desde=customer_date_desde, hasta = todays_date, tipo_negocio=1)

In [443]:
data_ventas = data_ventas[~data_ventas['sku_id'].isin(skus_remove)]

In [444]:
data_ventas = data_ventas[data_ventas['sku_id'].isin(good_skus)]

In [445]:
#//customers for offers */(has further uses)*/
microsegmentacion_df = segmentos_full[(~segmentos_full['customer_id'].isin(users_eliminar))]
data_ventas = data_ventas[(~data_ventas['customer_id'].isin(users_eliminar))]

### Create the MS "Other" for BAU 

In [446]:
data_full = data_ventas.merge(microsegmentacion_df[['customer_id','microsegment']], how = 'left', on = 'customer_id')

In [447]:
data_full['microsegment'] = data_full['microsegment'].fillna('Other')

In [448]:
data_full[['customer_id','microsegment']].groupby(['microsegment']).nunique().reset_index()

,microsegment,customer_id
0,Cafetería,649
1,Comida especializada,1004
2,Comida rápida,1601
3,Fonda,3369
4,Juguería/Comida saludable,1204
5,Other,932
6,Panadería/Pastelería,1595
7,Taquería,3260


In [449]:
segments_validation = data_full[['customer_id','microsegment']].groupby(['microsegment']).nunique().reset_index()

In [450]:
for ms in segments_validation[segments_validation.customer_id < 180]['microsegment'].unique():
    data_full['microsegment'] = data_full['microsegment'].str.replace(ms,'Other')

In [451]:
data_full[['customer_id','microsegment']].groupby(['microsegment']).nunique().reset_index()

,microsegment,customer_id
0,Cafetería,649
1,Comida especializada,1004
2,Comida rápida,1601
3,Fonda,3369
4,Juguería/Comida saludable,1204
5,Other,932
6,Panadería/Pastelería,1595
7,Taquería,3260


In [452]:
#Get the same variable to do the process job
data_ventas = data_full

### PARAMETERS // SOME FUNCTIONS - GET READY TO PROCESSING

In [453]:
def ms_data(city_code):

    from procurement_lib import redash, dw, send_message, send_slack_notification, GoogleSheet, logging
    
    paramss_csv=GoogleSheet("1k0ckgkLddxc5lY8NUVGCkvDLTuulLaJm-N535oMNZNw")
    city_paramss = paramss_csv.get_as_dataframe(city_code)
    
    parametros = {}
    parametros[city_code] = {}
    
    for i,ms in enumerate(city_paramss.iloc[:,4:].columns):
        parametros[city_code][ms] = [(value[1],value[2],value[3],value[4+i]) for value in city_paramss.values if value[4+i]>0]
        
    return parametros

In [464]:
parametros = ms_data(city_code)

In [465]:
def normalize_text(raw_text):
    """
    Removes common accent characters and normalize text.
    """
    raw_text = raw_text.replace(" ", "")
    raw_text = raw_text.replace("�", "")
    raw_text = raw_text.replace("/", "")
    raw_text = re.sub(r'\s*', '', raw_text)
    raw_text = re.sub(r"[àáâãäå]", 'a', raw_text)
    raw_text = re.sub(r"[èéêë]", 'e', raw_text)
    raw_text = re.sub(r"[ìíîï]", 'i', raw_text) 
    raw_text = re.sub(r"[òóôõö]", 'o', raw_text)
    raw_text = re.sub(r"[ùúûü]", 'u', raw_text)
    raw_text = re.sub(r"[ýÿ]", 'y', raw_text)
    raw_text = re.sub(r"[ß]", 'ss', raw_text)
    raw_text = re.sub(r"[ñ]", 'n', raw_text)
    raw_text = re.sub(r"[ç]", 'c', raw_text)
    raw_text = raw_text.lower()
    return raw_text

In [466]:
#Sacar de aca los microsegmentos
msg = data_full.microsegment.unique()
msg

array(['Taquería', 'Comida rápida', 'Fonda', 'Juguería/Comida saludable',
       'Panadería/Pastelería', 'Other', 'Comida especializada',
       'Cafetería'], dtype=object)

### END

In [467]:
#//Variables to store all the segments and offers of the city
global_offer_csv = pd.DataFrame()
global_segment_list = [] #//for procesing

In [468]:

#//----------------------------------------------------------------------------
#//Big loop for creating each city segments/offers files
for microsegment in msg:
    print('//-------------------------------------------------------------------------------')
    print(microsegment)

    #//Forced sku per microsegment
    forced_skus_formated = forced_skus[forced_skus['microsegment']==microsegment].groupby('subcat')['sku_ids'].unique().to_dict()
    #//Customers per microsegment
    customers_ms = data_ventas[(data_ventas['microsegment'] == microsegment)]['customer_id'].unique()
    #//Customers to give them the skus
    customers_for_skus_ms =  data_ventas[['microsegment','customer_id']][(data_ventas.microsegment == microsegment) & (~data_ventas.customer_id.isin(users_eliminar))]['customer_id'].unique()
    
    #----------------------------------------------------------- START OF GETTING READY THE DATA --------------------------------------------------------------------------------------
    
    
    #//----------------------------------------------------------------------------------------------------------------------------
    ms_name = normalize_text(microsegment) #NORMALIZE TEXT FORMAT
    #//----------------------------------------------------------------------------------------------------------------------------
    
    data_buyers = data_ventas[data_ventas['customer_id'].isin(customers_ms)].copy()
    print(data_buyers.customer_id.nunique())
    data_skus = data_ventas[data_ventas['customer_id'].isin(customers_for_skus_ms)].copy()


    subcats = list(map(lambda x: x[0], parametros[city_code][microsegment]))
    max_uses = list(map(lambda x: x[1], parametros[city_code][microsegment]))
    max_order = list(map(lambda x: x[2], parametros[city_code][microsegment]))
    discount = list(map(lambda x: x[3], parametros[city_code][microsegment]))

    #----------------------------------------------------------- END OF GETTING READY THE DATA --------------------------------------------------------------------------------------
    #----------------------------------------------------------- START OF THE GENERATION OF OFFERS --------------------------------------------------------------------------------------
    a, b, c = proceso_para_csv(data_ventas=data_buyers, todays_date=todays_date, 
                non_buyers_date=non_buyers_date, city=city_code, list_to_filter=subcats, offer_discount=discount, offer_max_uses=max_uses, offer_max_order=max_order, level='subcat', additional_words=ms_name,
                          limite=0.75, data_ventas_skus = data_skus, forzar_skus = forced_skus_formated)
    
    print(b.nunique())
    #//appending to global data frame
    global_offer_csv = global_offer_csv.append(a)
    
    #//Data formating
    lst = []
    name = 'export_data/MS/nb_'+city_code+'_'+ms_name+'.txt'
    for seg in b.columns:
        id_list = b[seg].dropna().astype(int)
        d = {'name':seg, 'customersIds': list(id_list)}
        lst.append(d)
        
           
    #//appending to global list    
        global_segment_list.append(d)
    
    print(f'Done {city_code} {ms_name}')
    #----------------------------------------------------------- END OF THE GENERATION OF OFFERS --------------------------------------------------------------------------------------
    
#//End of big loop and start of data bundle    

//-------------------------------------------------------------------------------
Taquería
3260
Uso de data externa
20230101_CMX_taqueria_NB_azucaryendulzantes_W52                2103
20230101_CMX_taqueria_NB_harinasymezclas_W52                   2915
20230101_CMX_taqueria_NB_aceitesygrasas_W52                    2192
20230101_CMX_taqueria_NB_cafechocolateeinfusiones_W52          3000
20230101_CMX_taqueria_NB_enlatados_W52                         2830
20230101_CMX_taqueria_NB_salsas_W52                            2782
20230101_CMX_taqueria_NB_arroz_W52                             2616
20230101_CMX_taqueria_NB_frutossecos_W52                       2826
20230101_CMX_taqueria_NB_panaderiaytortilleria_W52             2989
20230101_CMX_taqueria_NB_granos_W52                            2862
20230101_CMX_taqueria_NB_pasta_W52                             2815
20230101_CMX_taqueria_NB_salysazonadores_W52                   2538
20230101_CMX_taqueria_NB_chilessecos_W52                       2458


In [469]:
# #//--------------------------------------------
# #//Exporting single file data
# global_offer_csv.to_csv(f'SD/export_data/MS/csv_{city_code}_All_MS.csv', index = False)

# with open(f'SD/export_data/MS/nb_{city_code}_All_MS.txt', 'w') as outfile:
#         json.dump(global_segment_list, outfile)    
# with open(f'SD/export_data/MS/nb_{city_code}_All_MS.json', 'w') as outfile:
#         json.dump(global_segment_list, outfile)
# print(f'Done {city_code}')

# Aca Empieza la Experimentacion

In [470]:
from sklearn.model_selection import train_test_split 

In [471]:
#//Variables to store all the segments and offers of the city
global_offer_csv = pd.DataFrame()
global_segment_list = [] #//for procesing

In [472]:
#//----------------------------------------------------------------------------
#//Big loop for creating each city segments/offers files
aux_df = pd.DataFrame()
aux_df['aux']=5000*[0]
for microsegment in msg:
    print('//-------------------------------------------------------------------------------')
    print(microsegment)

    #//Forced sku per microsegment
    forced_skus_formated = forced_skus[forced_skus['microsegment']==microsegment].groupby('subcat')['sku_ids'].unique().to_dict()
    #//Customers per microsegment
    customers_ms = data_ventas[(data_ventas['microsegment'] == microsegment)]['customer_id'].unique()
    #//Customers to give them the skus
    customers_for_skus_ms =  data_ventas[['microsegment','customer_id']][(data_ventas.microsegment == microsegment) & (~data_ventas.customer_id.isin(users_eliminar))]['customer_id'].unique()
    
    #----------------------------------------------------------- START OF GETTING READY THE DATA --------------------------------------------------------------------------------------
    
    
    #//----------------------------------------------------------------------------------------------------------------------------
    ms_name = normalize_text(microsegment)+"_PILOTO" #NORMALIZE TEXT FORMAT
    #//----------------------------------------------------------------------------------------------------------------------------
    
    #// Separation of types of sales (non_buyers/skus for offers)
    #//////////////////////////////// THIS IS FOR CREATE EXPERIMENTS //////////////////////////////////////////////
    customer_ms_2, customer_ms_1 = train_test_split(customers_ms, test_size = 0.80, random_state=4)
    
    while abs(1-(data_ventas[data_ventas['customer_id'].isin(customer_ms_2)]['gmv_usd'].mean()/data_ventas[data_ventas['customer_id'].isin(customer_ms_1)]['gmv_usd'].mean())) > 0.05:
        #print(data_ventas[data_ventas['customer_id'].isin(customer_ms_1)]['gmv_usd'].mean(),data_ventas[data_ventas['customer_id'].isin(customer_ms_2)]['gmv_usd'].mean(),abs(1-(data_ventas[data_ventas['customer_id'].isin(customer_ms_2)]['gmv_usd'].mean()/data_ventas[data_ventas['customer_id'].isin(customer_ms_1)]['gmv_usd'].mean())))
        customer_ms_2, customer_ms_1 = train_test_split(customers_ms, test_size = 0.80)
    #//////////////////////////////// THIS IS FOR CREATE EXPERIMENTS //////////////////////////////////////////////
    
    data_buyers = data_ventas[data_ventas['customer_id'].isin(customer_ms_1)].copy() #le quite la s!!
    
    print(data_buyers.customer_id.nunique())
    data_skus = data_ventas[data_ventas['customer_id'].isin(customers_for_skus_ms)].copy()
    
    subcats = list(map(lambda x: x[0], parametros[city_code][microsegment]))
    max_uses = list(map(lambda x: int(x[1]), parametros[city_code][microsegment]))
    max_order = list(map(lambda x: int(x[2]), parametros[city_code][microsegment]))
    discount = list(map(lambda x: int(x[3]), parametros[city_code][microsegment]))

    #----------------------------------------------------------- END OF GETTING READY THE DATA --------------------------------------------------------------------------------------
    #----------------------------------------------------------- START OF THE GENERATION OF OFFERS --------------------------------------------------------------------------------------
    a, b, c = proceso_para_csv(data_ventas=data_buyers, todays_date=todays_date, 
            non_buyers_date=non_buyers_date, city=city_code, list_to_filter=subcats, offer_discount=discount, offer_max_uses=max_uses, offer_max_order=max_order, level='subcat', additional_words=ms_name,
                        limite=0.75, data_ventas_skus = data_skus, forzar_skus = forced_skus_formated)

    print(b.nunique())
    #//appending to global data frame
    global_offer_csv = global_offer_csv.append(a)
    
    #//Data formating
    lst = []
    name = 'export_data/MS/nb_'+city_code+'_'+ms_name+'.txt'
    for seg in b.columns:
        id_list = b[seg].dropna().astype(int)
        d = {'name':seg, 'customersIds': list(id_list)}
        aux_df.loc[range(len(list(id_list))),seg] = list(id_list)

        lst.append(d)
        
    #//appending to global list    
        global_segment_list.append(d)
    
    print(f'Done {city_code} {ms_name}')

#----------------------------------------------------------- END OF THE GENERATION OF OFFERS --------------------------------------------------------------------------------------
    
#////////////-------------------------------------------Here We Start With The Second Group-----------------------------------------------------------------------------------------------------------------
#////////////-------------------------------------------Here We Start With The Second Group-----------------------------------------------------------------------------------------------------------------
#////////////-------------------------------------------Here We Start With The Second Group-----------------------------------------------------------------------------------------------------------------
#////////////-------------------------------------------Here We Start With The Second Group-----------------------------------------------------------------------------------------------------------------
#////////////-------------------------------------------Here We Start With The Second Group-----------------------------------------------------------------------------------------------------------------


#----------------------------------------------------------- START OF GETTING READY THE DATA --------------------------------------------------------------------------------------
    
    
    #//----------------------------------------------------------------------------------------------------------------------------
    ms_name = normalize_text(microsegment)+"_CONTROL" #NORMALIZE TEXT FORMAT
    #//----------------------------------------------------------------------------------------------------------------------------
    
    #// Separation of types of sales (non_buyers/skus for offers)
    #//////////////////////////////// THIS IS FOR CREATE EXPERIMENTS //////////////////////////////////////////////
    data_buyers = data_ventas[data_ventas['customer_id'].isin(customer_ms_2)].copy() #le quite la s!!
    #//////////////////////////////// THIS IS FOR CREATE EXPERIMENTS //////////////////////////////////////////////
    
    print('//-------------------------------------------------------------------------------')
    print(f'{microsegment} Experimento50%')
    print(data_buyers.customer_id.nunique())
    data_skus = data_ventas[data_ventas['customer_id'].isin(customers_for_skus_ms)].copy()

    #----------------------------------------------------------- END OF GETTING READY THE DATA --------------------------------------------------------------------------------------
    #----------------------------------------------------------- START OF THE GENERATION OF OFFERS --------------------------------------------------------------------------------------
    e, f, g = proceso_para_csv(data_ventas=data_buyers, todays_date=todays_date, 
                non_buyers_date=non_buyers_date, city=city_code, list_to_filter=subcats, offer_discount=discount, offer_max_uses=max_uses, offer_max_order=max_order, level='subcat', additional_words=ms_name,
                          limite=0.75, data_ventas_skus = data_skus, forzar_skus = forced_skus_formated)
      
    print(f.nunique())
    #//appending to global data frame
    global_offer_csv = global_offer_csv.append(e)
    
    #//Data formating
    lst = []
    name = 'export_data/MS/nb_'+city_code+'_'+ms_name+'.txt'
    for seg in f.columns:
        id_list = f[seg].dropna().astype(int)
        h = {'name':seg, 'customersIds': list(id_list)}
        aux_df.loc[range(len(list(id_list))),seg] = list(id_list)
        
        lst.append(h)
        
        
    #//appending to global list    
        global_segment_list.append(h)
    
    print(f'Done {city_code} {ms_name}')
    #----------------------------------------------------------- END OF THE GENERATION OF OFFERS --------------------------------------------------------------------------------------
    
#//End of big loop and start of data bundle 

//-------------------------------------------------------------------------------
Taquería
2608
Uso de data externa
20230101_CMX_taqueria_piloto_NB_azucaryendulzantes_W52                1655
20230101_CMX_taqueria_piloto_NB_harinasymezclas_W52                   2333
20230101_CMX_taqueria_piloto_NB_aceitesygrasas_W52                    1754
20230101_CMX_taqueria_piloto_NB_cafechocolateeinfusiones_W52          2405
20230101_CMX_taqueria_piloto_NB_enlatados_W52                         2257
20230101_CMX_taqueria_piloto_NB_salsas_W52                            2220
20230101_CMX_taqueria_piloto_NB_arroz_W52                             2100
20230101_CMX_taqueria_piloto_NB_frutossecos_W52                       2259
20230101_CMX_taqueria_piloto_NB_panaderiaytortilleria_W52             2399
20230101_CMX_taqueria_piloto_NB_granos_W52                            2298
20230101_CMX_taqueria_piloto_NB_pasta_W52                             2257
20230101_CMX_taqueria_piloto_NB_salysazonadores_W52        

/opt/tljh/user/lib/python3.7/site-packages/pandas/core/indexing.py:1684: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  self.obj[key] = infer_fill_value(value)


Uso de data externa
20230101_CMX_comidarapida_control_NB_azucaryendulzantes_W52                216
20230101_CMX_comidarapida_control_NB_harinasymezclas_W52                   238
20230101_CMX_comidarapida_control_NB_aceitesygrasas_W52                    226
20230101_CMX_comidarapida_control_NB_cafechocolateeinfusiones_W52          299
20230101_CMX_comidarapida_control_NB_enlatados_W52                         258
20230101_CMX_comidarapida_control_NB_salsas_W52                            235
20230101_CMX_comidarapida_control_NB_arroz_W52                             277
20230101_CMX_comidarapida_control_NB_frutossecos_W52                       283
20230101_CMX_comidarapida_control_NB_panaderiaytortilleria_W52             282
20230101_CMX_comidarapida_control_NB_granos_W52                            296
20230101_CMX_comidarapida_control_NB_pasta_W52                             281
20230101_CMX_comidarapida_control_NB_salysazonadores_W52                   282
20230101_CMX_comidarapida_contro

## Aca creo el archivo para R&D

In [473]:
# customers =  pd.DataFrame()
# for column,i in enumerate(aux_df.columns):
#     #if ('verduras' in i or 'frutas' in i or 'arroz' in i or 'aceitesygrasas' in i or 'harinasymezclas' in i or 'azucaryendulzantes' in i or 'pollo' in i or 'huevos' in i) and not 'frutasjugo' in i:
#     if ('verduras' in i or 'frutas' in i) and not 'frutasjugo' in i:
#         customers[i] = aux_df[i]

In [474]:
#customers.info()

In [475]:
#customers.iloc[:3000].to_csv(f'SD/export_data/MS/csv_{city_code}_CLIENTS_PRUEBA_R&D.csv', index = False)

In [476]:
#with open(f'SD/export_data/MS/nb_{city_code}_All_MS.txt', 'w') as outfile:
#        json.dump(global_segment_list, outfile)

# Aca tengo que volver a cargar el archivo de arriba y dejarlo como lista para traer lo otro

In [477]:
global_offer_csv.head()

,offer_name,offer_description,discount,automatically_consider_offer,start_date,end_date,max_uses_per_order,max_uses_per_customer,customer_segment_id,sku_id
0,20230101_CMX_taqueria_piloto_NB_azucaryendulza...,taqueria_piloto_NB_azucaryendulzantes,8,true,01-01-2023,31-01-2023,15,1,None,101175#66553#101151#119540#228491
1,20230101_CMX_taqueria_piloto_NB_harinasymezcla...,taqueria_piloto_NB_harinasymezclas,6,true,01-01-2023,31-01-2023,16,1,None,130734#114083#52891#313473#-303957
2,20230101_CMX_taqueria_piloto_NB_aceitesygrasas...,taqueria_piloto_NB_aceitesygrasas,10,true,01-01-2023,31-01-2023,20,1,None,101239#54288#-303954#117707#117709#368168#3484...
3,20230101_CMX_taqueria_piloto_NB_cafechocolatee...,taqueria_piloto_NB_cafechocolateeinfusiones,6,true,01-01-2023,31-01-2023,18,1,None,26031#377839#313251#464689#313259
4,20230101_CMX_taqueria_piloto_NB_enlatados_W52,taqueria_piloto_NB_enlatados,7,true,01-01-2023,31-01-2023,20,1,None,56058#359348#359346#359344#359342#134385#35949...


In [478]:
len_seg =int(len(global_segment_list)/10)
len_seg_list = [global_segment_list[:len_seg],global_segment_list[len_seg:len_seg*2],global_segment_list[len_seg*2:len_seg*3],global_segment_list[len_seg*3:len_seg*4],global_segment_list[len_seg*4:len_seg*5],global_segment_list[len_seg*5:len_seg*6],global_segment_list[len_seg*6:len_seg*7],global_segment_list[len_seg*7:len_seg*8],global_segment_list[len_seg*8:len_seg*9],global_segment_list[len_seg*9:]]

In [479]:
import requests
import json

def create_segments(segment_list):
    
    url = f"https://{city_code}.frubana.com/api/v1/segments/create?api-key=00_growth-team-zzz-qqq_001"

    payload = json.dumps(segment_list)


    headers = {
    'Cookie': 'AWSALB=WGKrKvA1CFOY0m92xSENGAseuE4LrcfQh9y8jK/loGBJdizNm+FvjOqUqVghxdfLkH6Jsa9sfWft2NELchGAP/LNZMXMYhrOE70qDnUuOHATpdJj1UZBMEUBSIN1; AWSALBCORS=WGKrKvA1CFOY0m92xSENGAseuE4LrcfQh9y8jK/loGBJdizNm+FvjOqUqVghxdfLkH6Jsa9sfWft2NELchGAP/LNZMXMYhrOE70qDnUuOHATpdJj1UZBMEUBSIN1; AWSALB=jLY02FPysvF1vTrKLhXbk3ibzmyiAqFJ1ezYVu5PjodkU2QmPHDpRJhb9CvOlOyQ1Ix0mJR1ig+eH9zwJ9i5EOtgsgx79NSROogQb2Ua6yLX/I3bJJ6xtcVO3puI; AWSALBCORS=jLY02FPysvF1vTrKLhXbk3ibzmyiAqFJ1ezYVu5PjodkU2QmPHDpRJhb9CvOlOyQ1Ix0mJR1ig+eH9zwJ9i5EOtgsgx79NSROogQb2Ua6yLX/I3bJJ6xtcVO3puI',
    'Content-Type': 'application/json'
    }

    response = requests.request("POST", url, headers=headers, data=payload)
    print(response.status_code)
    sgt = [(int(x[:6]),x[7:]) for x in list(map(str.strip, response.text[32:].strip('][').replace('"', '').split(',')))]
    time.sleep(70)
    return sgt

In [480]:
sgt=[]

for i in len_seg_list:
    sgt.append(create_segments(i))
    
sgt_end =[item for lista in sgt for item in lista]

200
200
200
200
200
200
200
200
200
200


In [481]:
for i in global_offer_csv.offer_name:
    for j,z in sgt_end:
        if z==i:
            global_offer_csv.loc[global_offer_csv.offer_name == i, ('customer_segment_id')] = j

In [482]:
global_offer_csv.head()

,offer_name,offer_description,discount,automatically_consider_offer,start_date,end_date,max_uses_per_order,max_uses_per_customer,customer_segment_id,sku_id
0,20230101_CMX_taqueria_piloto_NB_azucaryendulza...,taqueria_piloto_NB_azucaryendulzantes,8,true,01-01-2023,31-01-2023,15,1,111929,101175#66553#101151#119540#228491
1,20230101_CMX_taqueria_piloto_NB_harinasymezcla...,taqueria_piloto_NB_harinasymezclas,6,true,01-01-2023,31-01-2023,16,1,111930,130734#114083#52891#313473#-303957
2,20230101_CMX_taqueria_piloto_NB_aceitesygrasas...,taqueria_piloto_NB_aceitesygrasas,10,true,01-01-2023,31-01-2023,20,1,111932,101239#54288#-303954#117707#117709#368168#3484...
3,20230101_CMX_taqueria_piloto_NB_cafechocolatee...,taqueria_piloto_NB_cafechocolateeinfusiones,6,true,01-01-2023,31-01-2023,18,1,111931,26031#377839#313251#464689#313259
4,20230101_CMX_taqueria_piloto_NB_enlatados_W52,taqueria_piloto_NB_enlatados,7,true,01-01-2023,31-01-2023,20,1,111933,56058#359348#359346#359344#359342#134385#35949...


In [483]:
global_offer_csv = global_offer_csv.reset_index(drop=True)
global_offer_csv.loc[global_offer_csv['offer_name'].str.contains("control", case=False), ('discount','max_uses_per_order')]=1
global_offer_csv.loc[global_offer_csv['offer_name'].str.contains("control", case=False), ('start_date','end_date')]='01-01-2023'
global_offer_csv.loc[global_offer_csv['offer_name'].str.contains("control", case=False), ('end_date')]='02-01-2023'
global_offer_csv.loc[global_offer_csv['offer_name'].str.contains("control", case=False), ('sku_id')]= global_offer_csv.loc[~global_offer_csv['offer_name'].str.contains("piloto", case=False), ('sku_id')].tolist()

In [484]:
len_offer =int(len(global_offer_csv)/4)
len_offer = [global_offer_csv[:len_seg],global_offer_csv[len_seg:len_seg*2],global_offer_csv[len_seg*2:len_seg*3],global_offer_csv[len_seg*3:]]

In [485]:
global_offer_csv.head()

,offer_name,offer_description,discount,automatically_consider_offer,start_date,end_date,max_uses_per_order,max_uses_per_customer,customer_segment_id,sku_id
0,20230101_CMX_taqueria_piloto_NB_azucaryendulza...,taqueria_piloto_NB_azucaryendulzantes,8,true,01-01-2023,31-01-2023,15,1,111929,101175#66553#101151#119540#228491
1,20230101_CMX_taqueria_piloto_NB_harinasymezcla...,taqueria_piloto_NB_harinasymezclas,6,true,01-01-2023,31-01-2023,16,1,111930,130734#114083#52891#313473#-303957
2,20230101_CMX_taqueria_piloto_NB_aceitesygrasas...,taqueria_piloto_NB_aceitesygrasas,10,true,01-01-2023,31-01-2023,20,1,111932,101239#54288#-303954#117707#117709#368168#3484...
3,20230101_CMX_taqueria_piloto_NB_cafechocolatee...,taqueria_piloto_NB_cafechocolateeinfusiones,6,true,01-01-2023,31-01-2023,18,1,111931,26031#377839#313251#464689#313259
4,20230101_CMX_taqueria_piloto_NB_enlatados_W52,taqueria_piloto_NB_enlatados,7,true,01-01-2023,31-01-2023,20,1,111933,56058#359348#359346#359344#359342#134385#35949...


In [486]:
#//--------------------------------------------
#//Exporting single file data
global_offer_csv.to_csv(f'SD/export_data/MS/csv_{city_code}_All_MS.csv', index = False)

for i,offer in enumerate(len_offer):
    offer.to_csv(f'SD/export_data/MS/csv_{city_code}_All_MS_{i}.csv', index = False)

with open(f'SD/export_data/MS/nb_{city_code}_All_MS.txt', 'w') as outfile:
        json.dump(global_segment_list, outfile)    
with open(f'SD/export_data/MS/nb_{city_code}_All_MS.json', 'w') as outfile:
        json.dump(global_segment_list, outfile)
print(f'Done {city_code}')

Done CMX


# Esto es el experimento de R&D

In [ ]:
# import ast

In [ ]:
# with open('SD/export_data/MS/nb_BOG_All_MS.txt', 'r') as file:
#     data = file.read()
# data = ast.literal_eval(data)

In [ ]:
# global_segment_list = data

In [ ]:
# exp_ryd_1 = pd.read_csv(f"csv_{city_code}_CLIENTS_FINAL_ENERO_R&D-other_b.csv")#['costumer_id'].unique().tolist()
# exp_ryd_2 = pd.read_csv(f"csv_{city_code}_CLIENTS_FINAL_ENERO_R&D-other_a.csv")#['costumer_id'].unique().tolist()
# exp_ryd_3 = pd.read_csv(f"csv_{city_code}_CLIENTS_FINAL_ENERO_R&D_fruver.csv")#['costumer_id'].unique().tolist()

# exp_ryd_4 = pd.concat([exp_ryd_1, exp_ryd_2], ignore_index=True)

# exp_ryd = pd.concat([exp_ryd_3, exp_ryd_4], ignore_index=True)

In [ ]:
# exp_ryd['previous_group'] = exp_ryd.previous_group.replace("Treatment","piloto")
# exp_ryd['previous_group'] = exp_ryd.previous_group.replace("Control","control")

# exp_ryd['final_group'] = exp_ryd.final_group.replace("Treatment","piloto")
# exp_ryd['final_group'] = exp_ryd.final_group.replace("Control","control")
# exp_ryd['offer_old'] = "20230101_"+city_code+"_"+exp_ryd.ms+"_"+exp_ryd.previous_group+"_NB_"+exp_ryd.subcategory+"_W52"
# exp_ryd['offer_new'] = "20230101_"+city_code+"_"+exp_ryd.ms+"_"+exp_ryd.final_group+"_NB_"+exp_ryd.subcategory+"_W52"

# exp_ryd['customer_source_id'] = exp_ryd.customer_source_id.astype(int)

# delete_old = exp_ryd[exp_ryd.offer_old != exp_ryd.offer_new].groupby('offer_old')['customer_source_id'].apply(list).reset_index(name='new')
# add_new = exp_ryd[exp_ryd.offer_old != exp_ryd.offer_new].groupby('offer_new')['customer_source_id'].apply(list).reset_index(name='new')

In [ ]:
# for lis in delete_old.values:
#     for dic in global_segment_list:
#         if dic['name'] == lis[0]:
#             for adds in lis[1]:
#                 #print(dic['name'])
#                 try:
#                     dic['customersIds'].remove(adds)
#                 except:
#                     pass
#                 #print(dic['customersIds'])
#                 break
#             break
#     break

In [ ]:
# for lis in add_new.values:
#     for dic in global_segment_list:
#         if dic['name'] == lis[0]:
#             for adds in lis[1]:
#                 dic['customersIds'].append(adds)
#                 #print(dic['customersIds'])
#                 break
#             break
#     break